In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/asmolka/repos/exess/smolkaa.github.io/api/exospheres"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.azimuth_angle_v

```
[1] azimuth_angle_v(v::AbstractVector)
[2] azimuth_angle_v(v::LocalCartesianVelocity)
```

Calculates the azimuth angle of the launch velocity vector `v`, measured mathematically positive from x (east). The resulting angle is given in radians, [-π, π).

Returns `NaN` if `v1=v2=0`, i.e., for no planar velocities.


In [3]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.eccentric_anomaly_e_M

```
[1] eccentric_anomaly_e_M(e::Real, M::Real, N=10)
```

Calculates the eccentric anomaly given the orbit's eccentricity `e` and the mean anomaly `M`,  which triggers a numerical approximation with `N` steps. The result is given in radians [0, 2π).

#### Alternative Functions

`eccentric_anomaly_e_theta`


In [4]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.eccentric_anomaly_e_theta

```
[1] eccentric_anomaly_e_theta(e::Real, theta::Real)
```

Calculates the eccentric anomaly given the orbit's eccentricity `e` and the true anomaly `theta`. The result is given in radians [0, 2π).

#### Alternative Functions

`eccentric_anomaly_e_M`


In [5]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.eccentricity_epskin_psi

```
[1] eccentricity_epskin_psi(epskin::Real, psi::Real)
```

Calculates the eccentricity of an elliptical trajetory based on the squared escape velocity fraction `epskin`, and the zenith launch angle `psi`.

#### Alternative Functions

`eccentricity_epskin_v`, `eccentricity_r_m_v`


In [6]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.eccentricity_epskin_v

```
[1] eccentricity_epskin_v(epskin::Real, v::AbstractVector)
[2] eccentricity_epskin_v(epskin::Real, v::LocalCartesianVelocity)
```

Calculates the eccentricity of an elliptical trajetory based on the squared escape velocity fraction `epskin`, and the velocity vector `v`.

#### Alternative Functions

`eccentricity_epskin_psi`, `eccentricity_r_m_v`


In [7]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.eccentricity_r_m_v

```
[1] eccentricity_r_m_v(r::Real, m::Real, v::AbstractVector)
[2] eccentricity_r_m_v(r::Real, m::Real, v::LocalCartesianVelocity)
```

Calculates the eccentricity of an elliptical trajetory based on the orbital radius `r`, the body's mass `m`, and the velocity vector `v`.

#### Alternative Functions

`eccentricity_epskin_psi`, `eccentricity_epskin_v`


In [8]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.escape_velocity_r_m

```
[1] escape_velocity_r_m(r::Real, m::Real)
```

Calculates the magnitude of the escape velocity for an orbital radius `r` and body mass `m`.


In [9]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.flight_time_t0_t1

```
[1] flight_time_t0_t1(t0::Real, t1::Real)
```

Calculates the time of flight between the two orbital times `t0` and `t1`.

#### Alternative Functions

`flight_time_t0_P`, `flight_time_e_theta0_theta1_P`, `flight_time_e_theta0_P`


In [10]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.flight_time_t0_P

```
[1] flight_time_t0_P(t0::Real, P::Real)
```

Calculates the time of flight, given a symmetrical trajectory with a starting orbital time of `t0` and a landing orbital time of `P-t0`, with the orbital period `P`. 

#### Alternative Functions

`flight_time_t0_t1`, `flight_time_e_theta0_theta1_P`, `flight_time_e_theta0_P`


In [11]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.flight_time_e_theta0_theta1_P

```
[1] flight_time_e_theta0_theta1_P(e::Real, theta0::Real, theta1::Real, P::Real)
```

Calculates the time of flight between the two orbital positions `theta0` and `theta1`, given the eccentricity `e` and period `P` of the orbit.

#### Alternative Functions

`flight_time_t0_t1`, `flight_time_t0_P`, `flight_time_e_theta0_P`


In [12]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.flight_time_e_theta0_P

```
[1] flight_time_e_theta0_P(e::Real, theta0::Real, P::Real)
```

Calculates the time of flight, given a symmetrical trajectory between the two orbital  positions `theta0` and `2π - theta0`, given the eccentricity `e` and period `P` of the orbit.

#### Alternative Functions

`flight_time_t0_t1`, `flight_time_t0_P`, `flight_time_e_theta0_theta1_P`


In [13]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.epskin_vesc_v

```
[1] epskin_vesc_v(vesc::Real, v::AbstractVector)
[2] epskin_vesc_v(vesc::Real, v::LocalCartesianVelocity)
```

Calculates the squared fraction of the velocity vector `v` with respect to the escape velocity `vesc`.

#### Alternative Functions

`epskin_r_m_v`, `epskin_r_a`, `epskin_e_theta`


In [14]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.epskin_r_m_v

```
[1] epskin_r_m_v(r::Real, m::Real, v::AbstractVector)
[2] epskin_r_m_v(r::Real, m::Real, v::LocalCartesianVelocity)
```

Calculates the squared fraction of the velocity vector `v`, based on the orbital radius  `r` and the body's mass `m`.

#### Alternative Functions

`epskin_vesc_v`, `epskin_r_a`, `epskin_e_theta`


In [15]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.epskin_r_a

```
[1] epskin_r_a(r::Real, a::Real)
```

Calculates the squared fraction of the velocity at the radial position `r` on an orbit defined through the semi-major axis `a`.

#### Alternative Functions

`epskin_vesc_v`, `epskin_r_m_v`, `epskin_e_theta`


In [16]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.epskin_e_theta

```
[1] epskin_e_theta(e::Real, theta::Real)
```

Calculates the squared fraction of the velocity at the angular position `theta` on an orbit defined through the eccentricity `e`.

#### Alternative Functions

`epskin_vesc_v`, `epskin_r_m_v`, `epskin_r_a`


In [17]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.ground_distance_R_dtheta

```
[1] ground_distance_R_dtheta(R::Real, dtheta::Real)
```

Calculates the ground flight distance covered by an angle `dtheta` of an elliptical orbit,  which intersects with a body of radius `R`.

#### Alternative Functions

`ground_distance_R_a_e`


In [18]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.ground_distance_R_a_e

```
[1] ground_distance_R_a_e(R::Real, a::Real, e::Real)
```

Calculates the ground flight distance covered based on the radius of the body `R`, the  orbit's semi-major axis `a`, and the eccentricity `e`.

#### Alternative Functions

`ground_distance_R_dtheta`


In [19]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.mean_anomaly_e_E

```
[1] mean_anomaly_e_E(e::Real, E::Real)
```

Calculates the mean anomaly, given the orbit's eccentricity `e` and the eccentric anomaly `E`.

#### Alternative Functions

`mean_anomaly_e_theta`


In [20]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.mean_anomaly_e_theta

```
[1] mean_anomaly_e_theta(e::Real, theta::Real)
```

Calculates the mean anomaly, given the orbit's eccentricity `e` and the true anomaly `theta`.

#### Alternative Functions

`mean_anomaly_e_E`


In [21]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.orbit_period_a_m

```
[1] orbit_period_a_m(a::Real, m::Real)
```

Calculates the period of an elliptical orbit, based on the semi-major axis `a`, and the body's mass `m`.


In [22]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.orbit_time_M_P

```
[1] orbit_time_M_P(M::Real, P::Real)
```

Calculates the time at the position of an orbit defined through the mean anomaly `M`, as  well as the orbital period `P`.

#### Alternative Functions

`orbit_time_e_E_P`, `orbit_time_e_theta_P`


In [23]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.orbit_time_e_E_P

```
[1] orbit_time_e_E_P(e::Real, E::Real, P::Real)
```

Calculates the time at the position of an orbit defined through the eccentric anomaly `E`,  the eccentricity `e`, and the orbital period `P`.

#### Alternative Functions

`orbit_time_M_P`, `orbit_time_e_theta_P`


In [24]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.orbit_time_e_theta_P

```
[1] orbit_time_e_theta_P(e::Real, theta::Real, P::Real)
```

Calculates the time at the position of an orbit defined through the true anomaly `theta`,  the eccentricity `e`, and the orbital period `P`.

#### Alternative Functions

`orbit_time_M_P`, `orbit_time_e_E_P`


In [25]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_latus_rectum_a_e

```
[1] semi_latus_rectum_a_e(a::Real, e::Real)
```

Calculates the semi-latus-rectum of the elliptical trajetory defined through the semi-major axis `a` and the eccentricity `e`.

#### Alternative Functions

`semi_latus_rectum_r_e_theta`


In [26]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_latus_rectum_r_e_theta

```
[1] semi_latus_rectum_r_e_theta(r::Real, e::Real, theta::Real)
```

Calculates the semi-latus-rectum of the elliptical trajetory based on the orbital radius `r`, the eccentricity `e`, and the true anomaly `theta`

#### Alternative Functions

`semi_latus_rectum_a_e`


In [27]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_major_axis_r_epskin

```
[1] semi_major_axis_r_epskin(r::Real, epskin::Real)
```

Calculates the semi-major axis of an elliptical orbit with the current orbital  radius `r` and the trajectory's squared escape velocity fraction `epskin`.

#### Alternative Functions

`semi_major_axis_r_m_v`, `semi_major_axis_p_e`, `semi_major_axis_r_e_theta`


In [28]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_major_axis_r_m_v

```
[1] semi_major_axis_r_m_v(r::Real, m::Real, v::AbstractVector)
[2] semi_major_axis_r_m_v(r::Real, m::Real, v::LocalCartesianVelocity)
```

Calculates the semi-major axis of an elliptical orbit with the current orbital  radius `r`, the body's mass `m`, and the velocity vector `v`.

#### Alternative Functions

`semi_major_axis_r_epskin`, `semi_major_axis_p_e`, `semi_major_axis_r_e_theta`


In [29]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_major_axis_p_e

```
[1] semi_major_axis_p_e(p::Real, e::Real)
```

Calculates the semi-major axis of an elliptical orbit with the semi-latus rectum `p` and  eccentricity `e`.

#### Alternative Functions

`semi_major_axis_r_epskin`, `semi_major_axis_r_m_v`, `semi_major_axis_r_e_theta`


In [30]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.semi_major_axis_r_e_theta

```
[1] semi_major_axis_r_e_theta(r::Real, e::Real, theta::Real)
```

Calculates the semi-major axis of an elliptical orbit with the oribital radius `r`, the eccentricity `e`, and the true anomaly `theta`.

#### Alternative Functions

`semi_major_axis_r_epskin`, `semi_major_axis_r_m_v`, `semi_major_axis_p_e`


In [31]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.true_anomaly_r_a_e

```
[1] true_anomaly_r_a_e(r::Real, a::Real, e::Real)
```

Calculates the true anomaly at the radial position `r` based on the semi-major axis `a`, and the eccentricity of the orbit `e`.

#### Alternative Functions

`true_anomaly_e_epskin`, `true_anomaly_epskin_v`


In [32]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.true_anomaly_e_epskin

```
[1] true_anomaly_e_epskin(e::Real, epskin::Real)
```

Calculates the true anomaly based on the eccentricity of the orbit `e` and the current squared escape velocity fraction `epskin`.

#### Alternative Functions

`true_anomaly_r_a_e`, `true_anomaly_epskin_v`


In [33]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.true_anomaly_epskin_v

No documentation found.

`Main.ExESS.true_anomaly_epskin_v` is a `Function`.

```
# 2 methods for generic function "true_anomaly_epskin_v" from Main.ExESS:
 [1] true_anomaly_epskin_v(epskin::Real, v::AbstractVector)
     @ ~/repos/exess/exess.jl/src/exospheres/orbital_mechanics.jl:426
 [2] true_anomaly_epskin_v(epskin::Real, v::LocalCartesianVelocity)
     @ ~/repos/exess/exess.jl/src/exospheres/orbital_mechanics.jl:427
```


In [34]:
#| echo: false
if !isdefined(Main, :ExESS)
include(joinpath(@__DIR__, "..", "..", "..", "exess.jl", "src", "ExESS.jl"))
using .ExESS
end
@doc ExESS.zenith_angle_v

```
[1] zenith_angle_v(v::AbstractVector)
[2] zenith_angle_v(v::LocalCartesianVelocity)
```

Calculates the zenith launch angle based on the launch velocity vector `v`.
